# Self-driving car Nanodegree - Term 1

## Project 2: **Behavior Cloning** 
***
In this project, we will use deep neural networks and convolutional neural networks to clone driving behavior. The model will output a steering angle to an autonomous vehicle.

#### Importing packages

In [1]:
import csv
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, Cropping2D, Lambda, Dropout
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
from keras.layers.pooling import MaxPooling2D
# Visualizations will be shown in the notebook.
%matplotlib inline

Using TensorFlow backend.


#### Loading the data

In [6]:
'''Read data'''
image_path = '../../../data'
driving_log_path = '../../../data/driving_log.csv'

rows = []
with open(driving_log_path) as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        rows.append(row)

def append_image(col, images, measurement, steering_measurements):
    current_path = image_path + '/' + col.strip()
    image = cv2.imread(current_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)
    steering_measurements.append(measurement)
    image_flipped = np.fliplr(image)
    images.append(image_flipped)
    measurement_flipped = measurement * (-1)
    steering_measurements.append(measurement)
          
images = []
steering_measurements = []
for line in rows[1:]:
    measurement = float(line[3])
    #center
    col_center = line[0]
    append_image(col_center, images, measurement, steering_measurements)
    #left
    col_left = line[1]
    append_image(col_left, images, measurement + 0.2, steering_measurements)
    #right
    col_right = line[2]
    append_image(col_right, images, measurement - 0.2, steering_measurements)

X_train = np.array(images)
y_train = np.array(steering_measurements)

IMG/center_2016_12_01_13_30_48_287.jpg
../../../data/IMG/center_2016_12_01_13_30_48_287.jpg
../../../data/IMG/left_2016_12_01_13_30_48_287.jpg
../../../data/IMG/right_2016_12_01_13_30_48_287.jpg
IMG/center_2016_12_01_13_30_48_404.jpg
../../../data/IMG/center_2016_12_01_13_30_48_404.jpg
../../../data/IMG/left_2016_12_01_13_30_48_404.jpg
../../../data/IMG/right_2016_12_01_13_30_48_404.jpg
IMG/center_2016_12_01_13_31_12_937.jpg
../../../data/IMG/center_2016_12_01_13_31_12_937.jpg
../../../data/IMG/left_2016_12_01_13_31_12_937.jpg
../../../data/IMG/right_2016_12_01_13_31_12_937.jpg
IMG/center_2016_12_01_13_31_13_037.jpg
../../../data/IMG/center_2016_12_01_13_31_13_037.jpg
../../../data/IMG/left_2016_12_01_13_31_13_037.jpg
../../../data/IMG/right_2016_12_01_13_31_13_037.jpg
IMG/center_2016_12_01_13_31_13_177.jpg
../../../data/IMG/center_2016_12_01_13_31_13_177.jpg
../../../data/IMG/left_2016_12_01_13_31_13_177.jpg
../../../data/IMG/right_2016_12_01_13_31_13_177.jpg
IMG/center_2016_12_01_13_

../../../data/IMG/left_2016_12_01_13_32_43_558.jpg
../../../data/IMG/right_2016_12_01_13_32_43_558.jpg
IMG/center_2016_12_01_13_32_43_659.jpg
../../../data/IMG/center_2016_12_01_13_32_43_659.jpg
../../../data/IMG/left_2016_12_01_13_32_43_659.jpg
../../../data/IMG/right_2016_12_01_13_32_43_659.jpg
IMG/center_2016_12_01_13_32_43_761.jpg
../../../data/IMG/center_2016_12_01_13_32_43_761.jpg
../../../data/IMG/left_2016_12_01_13_32_43_761.jpg
../../../data/IMG/right_2016_12_01_13_32_43_761.jpg
IMG/center_2016_12_01_13_32_43_862.jpg
../../../data/IMG/center_2016_12_01_13_32_43_862.jpg
../../../data/IMG/left_2016_12_01_13_32_43_862.jpg
../../../data/IMG/right_2016_12_01_13_32_43_862.jpg
IMG/center_2016_12_01_13_32_43_963.jpg
../../../data/IMG/center_2016_12_01_13_32_43_963.jpg
../../../data/IMG/left_2016_12_01_13_32_43_963.jpg
../../../data/IMG/right_2016_12_01_13_32_43_963.jpg
IMG/center_2016_12_01_13_32_44_064.jpg
../../../data/IMG/center_2016_12_01_13_32_44_064.jpg
../../../data/IMG/left_20

../../../data/IMG/right_2016_12_01_13_32_49_109.jpg
IMG/center_2016_12_01_13_32_49_210.jpg
../../../data/IMG/center_2016_12_01_13_32_49_210.jpg
../../../data/IMG/left_2016_12_01_13_32_49_210.jpg
../../../data/IMG/right_2016_12_01_13_32_49_210.jpg
IMG/center_2016_12_01_13_32_49_311.jpg
../../../data/IMG/center_2016_12_01_13_32_49_311.jpg
../../../data/IMG/left_2016_12_01_13_32_49_311.jpg
../../../data/IMG/right_2016_12_01_13_32_49_311.jpg
IMG/center_2016_12_01_13_32_49_413.jpg
../../../data/IMG/center_2016_12_01_13_32_49_413.jpg
../../../data/IMG/left_2016_12_01_13_32_49_413.jpg
../../../data/IMG/right_2016_12_01_13_32_49_413.jpg
IMG/center_2016_12_01_13_32_49_514.jpg
../../../data/IMG/center_2016_12_01_13_32_49_514.jpg
../../../data/IMG/left_2016_12_01_13_32_49_514.jpg
../../../data/IMG/right_2016_12_01_13_32_49_514.jpg
IMG/center_2016_12_01_13_32_49_615.jpg
../../../data/IMG/center_2016_12_01_13_32_49_615.jpg
../../../data/IMG/left_2016_12_01_13_32_49_615.jpg
../../../data/IMG/right_2

../../../data/IMG/left_2016_12_01_13_32_54_875.jpg
../../../data/IMG/right_2016_12_01_13_32_54_875.jpg
IMG/center_2016_12_01_13_32_54_976.jpg
../../../data/IMG/center_2016_12_01_13_32_54_976.jpg
../../../data/IMG/left_2016_12_01_13_32_54_976.jpg
../../../data/IMG/right_2016_12_01_13_32_54_976.jpg
IMG/center_2016_12_01_13_32_55_078.jpg
../../../data/IMG/center_2016_12_01_13_32_55_078.jpg
../../../data/IMG/left_2016_12_01_13_32_55_078.jpg
../../../data/IMG/right_2016_12_01_13_32_55_078.jpg
IMG/center_2016_12_01_13_32_55_179.jpg
../../../data/IMG/center_2016_12_01_13_32_55_179.jpg
../../../data/IMG/left_2016_12_01_13_32_55_179.jpg
../../../data/IMG/right_2016_12_01_13_32_55_179.jpg
IMG/center_2016_12_01_13_32_55_279.jpg
../../../data/IMG/center_2016_12_01_13_32_55_279.jpg
../../../data/IMG/left_2016_12_01_13_32_55_279.jpg
../../../data/IMG/right_2016_12_01_13_32_55_279.jpg
IMG/center_2016_12_01_13_32_55_380.jpg
../../../data/IMG/center_2016_12_01_13_32_55_380.jpg
../../../data/IMG/left_20

../../../data/IMG/right_2016_12_01_13_33_00_438.jpg
IMG/center_2016_12_01_13_33_00_540.jpg
../../../data/IMG/center_2016_12_01_13_33_00_540.jpg
../../../data/IMG/left_2016_12_01_13_33_00_540.jpg
../../../data/IMG/right_2016_12_01_13_33_00_540.jpg
IMG/center_2016_12_01_13_33_00_640.jpg
../../../data/IMG/center_2016_12_01_13_33_00_640.jpg
../../../data/IMG/left_2016_12_01_13_33_00_640.jpg
../../../data/IMG/right_2016_12_01_13_33_00_640.jpg
IMG/center_2016_12_01_13_33_00_740.jpg
../../../data/IMG/center_2016_12_01_13_33_00_740.jpg
../../../data/IMG/left_2016_12_01_13_33_00_740.jpg
../../../data/IMG/right_2016_12_01_13_33_00_740.jpg
IMG/center_2016_12_01_13_33_00_841.jpg
../../../data/IMG/center_2016_12_01_13_33_00_841.jpg
../../../data/IMG/left_2016_12_01_13_33_00_841.jpg
../../../data/IMG/right_2016_12_01_13_33_00_841.jpg
IMG/center_2016_12_01_13_33_00_943.jpg
../../../data/IMG/center_2016_12_01_13_33_00_943.jpg
../../../data/IMG/left_2016_12_01_13_33_00_943.jpg
../../../data/IMG/right_2

...

../../../data/IMG/right_2016_12_01_13_46_13_504.jpg
IMG/center_2016_12_01_13_46_13_604.jpg
../../../data/IMG/center_2016_12_01_13_46_13_604.jpg
../../../data/IMG/left_2016_12_01_13_46_13_604.jpg
../../../data/IMG/right_2016_12_01_13_46_13_604.jpg
IMG/center_2016_12_01_13_46_13_705.jpg
../../../data/IMG/center_2016_12_01_13_46_13_705.jpg
../../../data/IMG/left_2016_12_01_13_46_13_705.jpg
../../../data/IMG/right_2016_12_01_13_46_13_705.jpg
IMG/center_2016_12_01_13_46_13_808.jpg
../../../data/IMG/center_2016_12_01_13_46_13_808.jpg
../../../data/IMG/left_2016_12_01_13_46_13_808.jpg
../../../data/IMG/right_2016_12_01_13_46_13_808.jpg
IMG/center_2016_12_01_13_46_13_911.jpg
../../../data/IMG/center_2016_12_01_13_46_13_911.jpg
../../../data/IMG/left_2016_12_01_13_46_13_911.jpg
../../../data/IMG/right_2016_12_01_13_46_13_911.jpg
IMG/center_2016_12_01_13_46_14_011.jpg
../../../data/IMG/center_2016_12_01_13_46_14_011.jpg
../../../data/IMG/left_2016_12_01_13_46_14_011.jpg
../../../data/IMG/right_2

../../../data/IMG/left_2016_12_01_13_46_19_114.jpg
../../../data/IMG/right_2016_12_01_13_46_19_114.jpg
IMG/center_2016_12_01_13_46_19_214.jpg
../../../data/IMG/center_2016_12_01_13_46_19_214.jpg
../../../data/IMG/left_2016_12_01_13_46_19_214.jpg
../../../data/IMG/right_2016_12_01_13_46_19_214.jpg
IMG/center_2016_12_01_13_46_19_314.jpg
../../../data/IMG/center_2016_12_01_13_46_19_314.jpg
../../../data/IMG/left_2016_12_01_13_46_19_314.jpg
../../../data/IMG/right_2016_12_01_13_46_19_314.jpg
IMG/center_2016_12_01_13_46_19_415.jpg
../../../data/IMG/center_2016_12_01_13_46_19_415.jpg
../../../data/IMG/left_2016_12_01_13_46_19_415.jpg
../../../data/IMG/right_2016_12_01_13_46_19_415.jpg
IMG/center_2016_12_01_13_46_19_518.jpg
../../../data/IMG/center_2016_12_01_13_46_19_518.jpg
../../../data/IMG/left_2016_12_01_13_46_19_518.jpg
../../../data/IMG/right_2016_12_01_13_46_19_518.jpg
IMG/center_2016_12_01_13_46_19_619.jpg
../../../data/IMG/center_2016_12_01_13_46_19_619.jpg
../../../data/IMG/left_20

IMG/center_2016_12_01_13_46_24_821.jpg
../../../data/IMG/center_2016_12_01_13_46_24_821.jpg
../../../data/IMG/left_2016_12_01_13_46_24_821.jpg
../../../data/IMG/right_2016_12_01_13_46_24_821.jpg
IMG/center_2016_12_01_13_46_24_923.jpg
../../../data/IMG/center_2016_12_01_13_46_24_923.jpg
../../../data/IMG/left_2016_12_01_13_46_24_923.jpg
../../../data/IMG/right_2016_12_01_13_46_24_923.jpg
IMG/center_2016_12_01_13_46_25_025.jpg
../../../data/IMG/center_2016_12_01_13_46_25_025.jpg
../../../data/IMG/left_2016_12_01_13_46_25_025.jpg
../../../data/IMG/right_2016_12_01_13_46_25_025.jpg
IMG/center_2016_12_01_13_46_25_128.jpg
../../../data/IMG/center_2016_12_01_13_46_25_128.jpg
../../../data/IMG/left_2016_12_01_13_46_25_128.jpg
../../../data/IMG/right_2016_12_01_13_46_25_128.jpg
IMG/center_2016_12_01_13_46_25_229.jpg
../../../data/IMG/center_2016_12_01_13_46_25_229.jpg
../../../data/IMG/left_2016_12_01_13_46_25_229.jpg
../../../data/IMG/right_2016_12_01_13_46_25_229.jpg
IMG/center_2016_12_01_13_

../../../data/IMG/left_2016_12_01_13_46_30_613.jpg
../../../data/IMG/right_2016_12_01_13_46_30_613.jpg
IMG/center_2016_12_01_13_46_30_715.jpg
../../../data/IMG/center_2016_12_01_13_46_30_715.jpg
../../../data/IMG/left_2016_12_01_13_46_30_715.jpg
../../../data/IMG/right_2016_12_01_13_46_30_715.jpg
IMG/center_2016_12_01_13_46_30_816.jpg
../../../data/IMG/center_2016_12_01_13_46_30_816.jpg
../../../data/IMG/left_2016_12_01_13_46_30_816.jpg
../../../data/IMG/right_2016_12_01_13_46_30_816.jpg
IMG/center_2016_12_01_13_46_30_917.jpg
../../../data/IMG/center_2016_12_01_13_46_30_917.jpg
../../../data/IMG/left_2016_12_01_13_46_30_917.jpg
../../../data/IMG/right_2016_12_01_13_46_30_917.jpg
IMG/center_2016_12_01_13_46_31_019.jpg
../../../data/IMG/center_2016_12_01_13_46_31_019.jpg
../../../data/IMG/left_2016_12_01_13_46_31_019.jpg
../../../data/IMG/right_2016_12_01_13_46_31_019.jpg
IMG/center_2016_12_01_13_46_31_120.jpg
../../../data/IMG/center_2016_12_01_13_46_31_120.jpg
../../../data/IMG/left_20

../../../data/IMG/left_2016_12_01_13_46_36_265.jpg
../../../data/IMG/right_2016_12_01_13_46_36_265.jpg
IMG/center_2016_12_01_13_46_36_294.jpg
../../../data/IMG/center_2016_12_01_13_46_36_294.jpg
../../../data/IMG/left_2016_12_01_13_46_36_294.jpg
../../../data/IMG/right_2016_12_01_13_46_36_294.jpg
IMG/center_2016_12_01_13_46_36_366.jpg
../../../data/IMG/center_2016_12_01_13_46_36_366.jpg
../../../data/IMG/left_2016_12_01_13_46_36_366.jpg
../../../data/IMG/right_2016_12_01_13_46_36_366.jpg
IMG/center_2016_12_01_13_46_36_396.jpg
../../../data/IMG/center_2016_12_01_13_46_36_396.jpg
../../../data/IMG/left_2016_12_01_13_46_36_396.jpg
../../../data/IMG/right_2016_12_01_13_46_36_396.jpg
IMG/center_2016_12_01_13_46_36_468.jpg
../../../data/IMG/center_2016_12_01_13_46_36_468.jpg
../../../data/IMG/left_2016_12_01_13_46_36_468.jpg
../../../data/IMG/right_2016_12_01_13_46_36_468.jpg
IMG/center_2016_12_01_13_46_36_496.jpg
../../../data/IMG/center_2016_12_01_13_46_36_496.jpg
../../../data/IMG/left_20

In [7]:
print("Number of image is: {}", len(X_train))
print("Number of measurement is: {}", len(y_train))

Number of image is: {} 48216
Number of measurement is: {} 48216


### First attempt
In [my first attempt](https://github.com/tranlyvu/autonomous-vehicle-projects/blob/master/Behavioral%20Cloning/src/first_attempt.py), I used 9-layers network from [end to end learning for self-driving cars](https://arxiv.org/pdf/1604.07316.pdf) by NVIDIA 

1. Pre-processing pipeline
    - Data augmentation: Flipping images
    - Normalization
    - Mean centering
2. NVIDIA original model
    
|Layer   |type    |output filter/neurons|
|--------|--------|--------|
|1       |conv    |24      |
|2       |conv    |36      |
|3       |conv    |48      |
|4       |conv    |64      |
|5       |conv    |64      |
|6       |flattern|1164    |
|7       |relu    |100     |
|8       |relu    |50      |
|9       |relu    |10      |
|10      |relu    |1       |

### Second attempt

However, I detected overfitting in my first attempt, and hence i tried to improved the mode in second model by using regulation, i.e dropout

1. Pre-processing pipeline
    - Data augmentation: Flipping images
    - Normalization
    - Mean centering
2. Modified NVIDIA model
    
|Layer   |type    |output filter/neurons|
|--------|--------|--------|
|1       |conv    |24      |
|        |max_pool|        |
|2       |conv    |36      |
|        |max_pool|        |
|3       |conv    |48      |
|        |max_pool|        |
|4       |conv    |64      |
|        |max_pool|        |
|5       |conv    |64      |
|6       |flattern|1164    |
|7       |relu    |100     |
|8       |relu    |50      |
|9       |relu    |10      |
|10      |relu    |1       |

In [8]:
model = Sequential()
#The cameras in the simulator capture 160 pixel by 320 pixel images., after cropping, it is 66x200
model.add(Cropping2D(cropping = ((74,20), (60,60)),input_shape=(160, 320, 3)))

model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(66, 200, 3)))
model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu'))
#model.add(Dropout(.5))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu'))
#model.add(Dropout(.5))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))  
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

#### Training

In [ ]:
'''Training: using MSE for regression'''
model.compile(loss='mse', optimizer='adam')
def generator(samples, batch_size = 32):
    num_samples = len(samples)
    while 1:
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset + batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                name = '../../../data/IMG/' + batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            x_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(x_train, y_train)
            
train_samples, validation_samples = train_test_split(rows[1:], test_size = 0.2)

train_generator = generator(train_samples, batch_size = 32)
validation_generator = generator(validation_samples, batch_size = 32)

history_object = model.fit_generator(train_generator,
                                    samples_per_epoch = len(train_samples),
                                    validation_data = validation_generator,
                                    nb_val_samples = len(validation_samples),
                                    nb_epoch = 40)

Epoch 1/40
5568/6428 [========================>.....] - ETA: 1s - loss: 0.0117

In [ ]:
model.save('../model.h5')
print(history_object.history.keys())
###plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()